In [1]:
print("ok")

ok


In [3]:
%pwd

'c:\\Users\\Nikita Ballav\\OneDrive\\Desktop\\Chat Clinic\\Chat-Clinic\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Nikita Ballav\\OneDrive\\Desktop\\Chat Clinic\\Chat-Clinic'

In [5]:
from  langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extract Data from the PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
# Split the data into text chunks

def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks= text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings= download_hugging_face_embeddings()

C:\Users\Nikita Ballav\AppData\Local\Temp\ipykernel_13376\2099866616.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Nikita Ballav\anaconda3\envs\chat_clinic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import sentence_transformers
print(sentence_transformers.__version__)


3.4.1


In [14]:
query_result=embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
Pinecone_API_Key = os.environ.get('Pinecone_API_Key')


In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=Pinecone_API_Key)

index_name="clinicvector"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [23]:
import os
os.environ["Pinecone_API_Key"] = Pinecone_API_Key



In [25]:
# embed each chunk and upsert the embeddings into your Pinecone index

from langchain_pinecone import PineconeVectorStore
index_name="clinicvector"
docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings

)

In [26]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding=embeddings
)

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs = retriever.invoke("what is acne?")

In [30]:
retrieved_docs

[Document(id='9eadd686-a306-433f-8719-89eed2ea7475', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='ec9837aa-4b53-4547-8ff0-11e1bfd49006', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nate

In [ ]:
# # pip install google-generativeai langchain-google-genai
# import google.generativeai as genai
# import os
# from langchain_google_genai import ChatGoogleGenerativeAI

# # Configuration
# genai.configure(api_key=os.getenv("Gemini_API_key"))
# generation_config={"temperature":0.9,"top_p":1,"top_k":1,"max_output_tokens":500}

# # Initialize the model
# model= genai.GenerativeModel("gemini-2.0-flash", generation_config=generation_config)

# # response
# response= model.generate_content("what is Acne?")

# # # Set API key explicitly
# # os.environ["Gemini_API_key"] = Gemini_API_key

# # # Initialize Gemini model
# # llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.4, max_output_tokens=500)


In [31]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4,max_tokens=500)

from langchain_community.llms import Ollama

# Define llm
llm = Ollama(model="deepseek-r1:7b")

C:\Users\Nikita Ballav\AppData\Local\Temp\ipykernel_13376\2491977304.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1:7b")


In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt= (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:

question_answer_chain= create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

<think>
Okay, so I need to figure out what Acne is based on the given context. Let me read through the passages carefully.

First passage mentions that Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when pores are clogged with oil, dead skin cells, and bacteria. The second passage provides more detail: Acne vulgaris affects women's faces and is caused by inflammation of sebaceous glands.

So putting it together, Acne is a skin condition with bumps, usually on the face, from blocked pores due to oil, dead skin, and bacteria. Acne vulgaris specifically affects women's faces because of sebaceous gland inflammation.
</think>

Acne is a common skin disease characterized by pitted red bumps (pimples) typically on the face, chest, or back, caused by clogged pores filled with oil, dead skin cells, and bacteria.
